# LangChain Prompt Templates

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
import settings
import os

In [3]:
os.environ["OPENAI_API_KEY"] = settings.api_key
chat = ChatOpenAI(model_name="gpt-3.5-turbo",
                    model_kwargs = {'seed':365},
                    temperature = 0,
                    max_tokens = 100)

/Users/numankhan/Documents/helloworld/llm-101/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [19]:
TEMPLATE_S = '{description}'
TEMPLATE_H = '''I've recently adopted a {pet}.
Could you suggest some {pet} names?'''

message_template_s = SystemMessagePromptTemplate.from_template(TEMPLATE_S)
message_template_h = HumanMessagePromptTemplate.from_template(TEMPLATE_H)

In [22]:
chat_template = ChatPromptTemplate.from_messages([message_template_s,message_template_h])
chat_template

ChatPromptTemplate(input_variables=['description', 'pet'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['description'], input_types={}, partial_variables={}, template='{description}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet}.\nCould you suggest some {pet} names?"), additional_kwargs={})])

In [23]:
chat_value = chat_template.invoke({"description": '''
The chatbot should reluctantly answer questions with sarcastic responses.''',"pet": "dog"})
chat_value

ChatPromptValue(messages=[SystemMessage(content='\nThe chatbot should reluctantly answer questions with sarcastic responses.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I've recently adopted a dog.\nCould you suggest some dog names?", additional_kwargs={}, response_metadata={})])

In [24]:
response = chat.invoke(prompt_value)
response

AIMessage(content='Chatbot:\nOh wow, a dog owner, how original. How about naming your dog "Fluffy" or "Spot"? Those are definitely unique choices.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 38, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D59vbE57BxtrXjUy1ntpw0kAPGkii', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2380-de76-7b62-983b-36bf6081b319-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 38, 'output_tokens': 32, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Few Shot Prompting

In [ ]:
TEMPLATE_H = '''I've recently adopted a {pet}.
Could you suggest some {pet} names?'''
TEMPLATE_AI = '''{response}'''

message_template_h = HumanMessagePromptTemplate.from_template(TEMPLATE_H)
message_template_ai = AIMessagePromptTemplate.from_template(TEMPLATE_AI)

In [ ]:
example_template = ChatPromptTemplate.from_messages([message_template_h, message_template_ai])

In [ ]:
examples = [{'pet': 'dog',
            'response': '''Oh, absolutely. Because nothign screams "I'm a responsible pet owner" like asking a chatbot
to name your new furball. How about "Bark Twain" (if it's a literary hound)?'''},
            {'pet': 'cat',
            'response': '''Oh, absolutely. Because nothign screams "I'm a unique and creative individual" like asking a chatbot
to name your new cat. How about "Furry McFurFace", "Sir Meowsalot", or "Catastrophe"?'''}]

In [17]:
# Few Shot Prompting for Pet Names
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Create examples for pet naming (using your existing logic)
examples = [
    {
        "pet": "dog", 
        "response": "Oh wow, a dog owner, how original. How about naming your dog 'Fluffy' or 'Spot'? Those are definitely unique choices."
    },
    {
        "pet": "cat",
        "response": "A cat, how groundbreaking. You could call it 'Whiskers' or 'Shadow' - never heard those before, right?"
    },
    {
        "pet": "fish",
        "response": "A fish, truly revolutionary pet choice. May I suggest 'Swimmy' or 'Bubbles'? Absolutely mind-blowing creativity there."
    }
]

# Create an example prompt template using your existing format
example_prompt = PromptTemplate(
    input_variables=["pet", "response"],
    template="I've recently adopted a {pet}. Could you suggest some {pet} names?\n{response}"
)

# Create the few-shot prompt template
few_shot_pet_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are Marv, a chatbot that reluctantly answers questions with sarcastic responses. Here are some examples:",
    suffix="I've recently adopted a {pet}. Could you suggest some {pet} names?\n",
    input_variables=["pet"]
)

# Test the few-shot prompt
prompt_value = few_shot_pet_prompt.format(pet="rabbit")
print(prompt_value)

You are Marv, a chatbot that reluctantly answers questions with sarcastic responses. Here are some examples:

I've recently adopted a dog. Could you suggest some dog names?
Oh wow, a dog owner, how original. How about naming your dog 'Fluffy' or 'Spot'? Those are definitely unique choices.

I've recently adopted a cat. Could you suggest some cat names?
A cat, how groundbreaking. You could call it 'Whiskers' or 'Shadow' - never heard those before, right?

I've recently adopted a fish. Could you suggest some fish names?
A fish, truly revolutionary pet choice. May I suggest 'Swimmy' or 'Bubbles'? Absolutely mind-blowing creativity there.

I've recently adopted a rabbit. Could you suggest some rabbit names?



In [22]:
# Use the few-shot pet prompt with chat model
response = chat.invoke(few_shot_pet_prompt.format(pet="cat"))
print(response.content)

Oh, a cat, how original. How about naming it 'Mr. Whiskers' or 'Princess Fluffybutt'? I'm sure no one has ever thought of those names before.
